## Import Packages

In [1]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn import metrics

In [2]:
random_seed = 8022022 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

## Read dataset

### run R script to generate data

In [3]:
# taxonomy_level = 'species','genus','family'
# phylotype = '1e_1','1e0','5e_1'

def datagenerator(taxonomy_level = 'species',taxonomy_hidden = 50, \
                  phylotype_level = '1e_1', phylotype_hidden = 50, outcome = "was_preterm"):
    
    ##define paths
    meta_path = 'data/meta_' + outcome + '.csv'
    
    tax_path = 'data/training_data_2022-07-21/taxonomy/taxonomy_relabd.' + taxonomy_level + '.csv'
    tax_profile_path = 'DeepMicrodata/data/taxonomy_' + taxonomy_level + '_' + outcome + '.csv'
    tax_log = 'DeepMicrodata/taxonomy_log_'+ taxonomy_level + '_' + outcome + '.txt'
    phylo_path = 'data/training_data_2022-07-21/phylotypes/phylotype_relabd.' + phylotype_level + '.csv'
    phylo_profile_path = 'DeepMicrodata/data/phylotype_' + phylotype_level + '_' + outcome + '.csv'
    phylo_log = 'DeepMicrodata/phylotype_log_'+ phylotype_level + '_' + outcome + '.txt'
    tax_hidden_path = 'DeepMicrodata/results/' + 'AE[' + str(taxonomy_hidden) + ']' + '_' + \
                os.path.splitext(os.path.basename(tax_profile_path))[0] + '_rep.csv'
    phylo_hidden_path = 'DeepMicrodata/results/' + 'AE[' + str(phylotype_hidden) + ']' + '_' + \
                os.path.splitext(os.path.basename(phylo_profile_path))[0] + '_rep.csv'
    
    ##generate data and run DeepMicro
    if not os.path.exists(tax_profile_path):
        !"C:/Program Files/R/R-4.2.0/bin/x64/Rscript.exe" --verbose --vanilla data_generator_no_filter.R \
        {outcome} {tax_path} {meta_path} {tax_profile_path} 

    if not os.path.exists(tax_hidden_path):
        !python DeepMicro/DM.py -r 1 -s 1000 -p DeepMicrodata/ --no_clf \
        -cd {os.path.basename(tax_profile_path)} --ae -dm {taxonomy_hidden} --save_rep > {tax_log}
    
    if not os.path.exists(phylo_profile_path):
        !"C:/Program Files/R/R-4.2.0/bin/x64/Rscript.exe" --verbose --vanilla data_generator_no_filter.R \
        {outcome} {phylo_path} {meta_path} {phylo_profile_path}
    
    if not os.path.exists(phylo_hidden_path):
        !python DeepMicro/DM.py -r 1 -s 1000 -p DeepMicrodata/ --no_clf \
        -cd {os.path.basename(phylo_profile_path)} --ae -dm {phylotype_hidden} --save_rep > {phylo_log}
    
    ##read data
    metadata = pd.read_csv(meta_path, delimiter=',')
    
    
    
    taxdata = pd.read_csv(tax_hidden_path, delimiter=',', header = None)
    
    taxname = ['tax' + str(i) for i in range(taxdata.shape[1])]
    
    taxdata.columns = taxname
    
    
    
    phylodata = pd.read_csv(phylo_hidden_path, delimiter=',', header = None)
    
    phyloname = ['phylo' + str(i) for i in range(phylodata.shape[1])]
    
    phylodata.columns = phyloname
    
    mydata = pd.concat([metadata,taxdata,phylodata], axis = 1).copy().set_index('specimen')
    
    
    return mydata
     


In [98]:
outcome = "was_preterm"

In [99]:
mydata = datagenerator(taxonomy_level = 'genus',taxonomy_hidden = 20, \
                  phylotype_level = '5e_1', phylotype_hidden = 30, outcome = outcome)

In [100]:
print(mydata.shape)
mydata.head(5)

(2077, 65)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,age_imp,CST,collect_wk,shannon,inv_simpson,...,phylo20,phylo21,phylo22,phylo23,phylo24,phylo25,phylo26,phylo27,phylo28,phylo29
specimen,,,,,,,,,,,,,,,,,,,,,
A00003-02,A00003,A,False,False,40,32,II,30,1.00000,1.00000,...,-0.192806,-0.243930,0.077513,-0.223137,0.054267,0.213074,0.135706,0.204574,-0.114253,-0.160474
A00004-08,A00004,A,False,False,40,25,III,27,1.00000,1.00000,...,-0.245921,0.032504,-0.154268,0.154259,-0.210851,0.130357,-0.263512,-0.281376,-0.128630,-0.238920
A00004-12,A00004,A,False,False,40,25,III,29,6.94884,4.07385,...,-0.211545,0.022500,-0.142456,0.138919,-0.146539,0.094257,-0.211134,-0.251107,-0.124423,-0.231162
A00004-13,A00004,A,False,False,40,25,III,30,3.04127,2.30305,...,-0.175474,0.019546,-0.125339,0.127339,-0.091471,0.062716,-0.160936,-0.226131,-0.112218,-0.214659
A00005-02,A00005,A,False,False,41,31,II,26,1.85087,1.79889,...,-0.090355,-0.204638,0.091833,0.154889,-0.015265,-0.133653,0.079635,-0.153158,0.073932,-0.075373


### change data type

In [101]:
mydata["project"] = mydata["project"].astype('category')
mydata["CST"] = mydata["CST"].astype('category')
mydata['was_preterm'] = mydata['was_preterm'].astype('int64')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int64')

mydata.dtypes

participant_id         object
project              category
was_preterm             int64
was_early_preterm       int64
delivery_wk             int64
                       ...   
phylo25               float64
phylo26               float64
phylo27               float64
phylo28               float64
phylo29               float64
Length: 65, dtype: object

### subset data by outcome

In [102]:
# only keep the rows with collect_wk < 32 for Preterm task
mydata_preterm_t = mydata.loc[mydata['collect_wk']<=32,].copy().sort_index(axis = 0)

mydata_preterm = mydata_preterm_t 
# mydata_preterm = mydata_preterm_t.loc[mydata_preterm_t['project']!= 'I',].copy().sort_index(axis = 0)
print(mydata_preterm.shape)

mydata_preterm.head(5)

(2077, 65)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,age_imp,CST,collect_wk,shannon,inv_simpson,...,phylo20,phylo21,phylo22,phylo23,phylo24,phylo25,phylo26,phylo27,phylo28,phylo29
specimen,,,,,,,,,,,,,,,,,,,,,
A00003-02,A00003,A,0,0,40,32,II,30,1.00000,1.00000,...,-0.192806,-0.243930,0.077513,-0.223137,0.054267,0.213074,0.135706,0.204574,-0.114253,-0.160474
A00004-08,A00004,A,0,0,40,25,III,27,1.00000,1.00000,...,-0.245921,0.032504,-0.154268,0.154259,-0.210851,0.130357,-0.263512,-0.281376,-0.128630,-0.238920
A00004-12,A00004,A,0,0,40,25,III,29,6.94884,4.07385,...,-0.211545,0.022500,-0.142456,0.138919,-0.146539,0.094257,-0.211134,-0.251107,-0.124423,-0.231162
A00004-13,A00004,A,0,0,40,25,III,30,3.04127,2.30305,...,-0.175474,0.019546,-0.125339,0.127339,-0.091471,0.062716,-0.160936,-0.226131,-0.112218,-0.214659
A00005-02,A00005,A,0,0,41,31,II,26,1.85087,1.79889,...,-0.090355,-0.204638,0.091833,0.154889,-0.015265,-0.133653,0.079635,-0.153158,0.073932,-0.075373


In [103]:
mydata_preterm_ind = mydata_preterm.groupby('participant_id').first().copy()

mydata_preterm_ind.shape

(1068, 64)

### split train validation id set

In [104]:
test_id = list(mydata_preterm_ind.groupby(outcome).sample(frac = 0.2, random_state = 200).sort_index(axis = 0).index)

# train_val_id = list(mydata_preterm_ind.loc[~mydata_preterm_ind.index.isin(train_id)].groupby('was_preterm').\
#             sample(frac = 0.5, random_state = 200).sort_index(axis = 0).index)

train_val_id = list(np.setdiff1d(list(mydata_preterm_ind.index),test_id))

print(len(test_id))
print(len(train_val_id))


213
855


### obtain train validation dataset

In [105]:
test_data = mydata_preterm[mydata_preterm['participant_id'].isin(test_id)]

print(len(test_id))
print(test_data.shape)
test_data.head(5)

213
(382, 65)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,age_imp,CST,collect_wk,shannon,inv_simpson,...,phylo20,phylo21,phylo22,phylo23,phylo24,phylo25,phylo26,phylo27,phylo28,phylo29
specimen,,,,,,,,,,,,,,,,,,,,,
A00004-08,A00004,A,0,0,40,25,III,27,1.00000,1.00000,...,-0.245921,0.032504,-0.154268,0.154259,-0.210851,0.130357,-0.263512,-0.281376,-0.128630,-0.238920
A00004-12,A00004,A,0,0,40,25,III,29,6.94884,4.07385,...,-0.211545,0.022500,-0.142456,0.138919,-0.146539,0.094257,-0.211134,-0.251107,-0.124423,-0.231162
A00004-13,A00004,A,0,0,40,25,III,30,3.04127,2.30305,...,-0.175474,0.019546,-0.125339,0.127339,-0.091471,0.062716,-0.160936,-0.226131,-0.112218,-0.214659
A00010-01,A00010,A,0,0,38,30,III,27,1.00000,1.00000,...,-0.150443,0.017638,-0.114856,0.107124,-0.140615,0.089013,-0.170773,-0.190238,-0.074177,-0.145844
A00010-02,A00010,A,0,0,38,30,III,28,2.06211,1.83220,...,-0.118695,0.013494,-0.098349,0.109109,-0.148815,0.088655,-0.147476,-0.174987,-0.048399,-0.108551


In [106]:
train_val_data = mydata_preterm[mydata_preterm['participant_id'].isin(train_val_id)]

print(len(train_val_id))
print(train_val_data.shape)
train_val_data.head(5)

855
(1695, 65)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,age_imp,CST,collect_wk,shannon,inv_simpson,...,phylo20,phylo21,phylo22,phylo23,phylo24,phylo25,phylo26,phylo27,phylo28,phylo29
specimen,,,,,,,,,,,,,,,,,,,,,
A00003-02,A00003,A,0,0,40,32,II,30,1.00000,1.00000,...,-0.192806,-0.243930,0.077513,-0.223137,0.054267,0.213074,0.135706,0.204574,-0.114253,-0.160474
A00005-02,A00005,A,0,0,41,31,II,26,1.85087,1.79889,...,-0.090355,-0.204638,0.091833,0.154889,-0.015265,-0.133653,0.079635,-0.153158,0.073932,-0.075373
A00005-05,A00005,A,0,0,41,31,II,29,1.00000,1.00000,...,-0.109795,-0.243072,0.118366,0.191589,-0.017404,-0.165640,0.095252,-0.185196,0.088213,-0.100065
A00006-01,A00006,A,0,0,41,28,II,31,1.00000,1.00000,...,-0.154215,-0.200532,0.062518,-0.167118,0.037809,0.164644,0.105818,0.130045,-0.091391,-0.113859
A00008-02,A00008,A,1,0,35,38,IV-B,17,1.00000,1.00000,...,-0.067387,-0.084873,0.045210,-0.029346,0.049795,0.040730,-0.025317,0.051671,-0.084296,0.062176


In [107]:
def tensor_generator(data, outcome = outcome):
    X_data = data[data.columns[6:]].copy()
    X_data['CST'] = X_data['CST'].cat.codes
    X_feature = X_data.to_numpy().astype('float32')
    X_group = data['participant_id'].astype('category').cat.codes.values.reshape(-1,1)
    input_X = torch.from_numpy(np.hstack((X_feature,X_group)))
    y_data = data.groupby('participant_id').first()
    y = y_data[outcome].values.flatten()
    input_y = torch.from_numpy(np.vstack((y,1-y)).T.astype('float32'))

    
    return input_X, input_y

    

In [108]:
# Define RNN model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1,hidden_dim2,drop_out):
        

        
        #inherit from super class
        super(MLP, self).__init__()
        
        #define layers
        
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1,hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2,2)
        self.dropout = nn.Dropout(drop_out)

        
    def forward(self, x):
        
        X_feature = x[:,:-1]
        X_group = x[:,-1].long()
        
        X_feature = torch.tanh(self.fc1(X_feature))
        X_feature = self.dropout(X_feature)
        X_feature = torch.tanh(self.fc2(X_feature))
        X_feature = self.dropout(X_feature)
        X_feature = torch.tanh(self.fc3(X_feature))

        X_feature = F.softmax(X_feature, dim = 1)

        
        M = torch.zeros(X_group.max()+1, len(X_feature))
        M[X_group, torch.arange(len(X_feature))] = 1
        M = F.normalize(M, p=1, dim=1)
        X_feature = torch.mm(M, X_feature)

        
        return X_feature
    

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01) 


In [109]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [110]:
def indloader(data, seed, outcome = outcome):
    data_ind = data.groupby('participant_id').first().copy()
    id_1 = list(data_ind.groupby(outcome).sample(frac = 0.5, random_state = seed).sort_index(axis = 0).index)
    id_2 = list(np.setdiff1d(list(data_ind.index),id_1))
    
    return [id_1,id_2]
    

In [111]:
def train_epoch(train_data, val_data, learn_rate, outcome = outcome, \
                hidden_dim1 = 10,hidden_dim2 = 10,drop_out = 0.1, \
                device = device, EPOCHS = 500, patience = 20):
    
    val_X, val_y = tensor_generator(val_data)
    
    
    #input_dim
    
    input_dim = val_X.shape[1]-1
    

    model = MLP(input_dim, hidden_dim1,hidden_dim2,drop_out)

        
    model.to(device)
    
    model.apply(init_weights)
    # loss criterion and optimizer

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
    
    train_loss = []
    val_loss = []
    # train model
    
    model.train()
    
    last_loss = 1000
    trigger_times = 0
    #Start training loop
    
    for epoch in range(1, EPOCHS + 1):
        
        batch_train_losses = []
        ind_loader = indloader(train_data, seed = 980 + epoch, outcome = outcome)
        
        
        for ind in ind_loader:
            
            train_batch = train_data[train_data['participant_id'].isin(ind)]
            
            x, label = tensor_generator(train_batch)
            
            x, label = x.to(device), label.to(device)
            
            model.zero_grad()
        
            predictions = model(x)
            
            predictions = predictions.to(device)
        
            loss = criterion(predictions, label)
            
            batch_train_losses.append(loss.detach().numpy())
        
            # backpropagation
            loss.backward() 
            # Updates the weights accordingly
            optimizer.step()
        
        train_loss.append(np.mean(batch_train_losses))
        
        with torch.no_grad():
            
            model.eval()
            
            val_X, val_y = val_X.to(device), val_y.to(device)
                
            predictions_val = model(val_X)
                
            predictions_val = predictions_val.to(device)
                
            loss_val = criterion(predictions_val, val_y)
            
            val_loss.append(loss_val.detach().numpy())
            

        current_loss = val_loss[epoch-1]
        
        
        if current_loss >= last_loss:
            trigger_times += 1
            
            if trigger_times >= patience:
                print ('Early stopping at {}'.format(epoch))
                return model,train_loss,val_loss
        else:
            tigger_times = 0
        
        last_loss = current_loss
        
        if epoch%10 == 0:
            print('Epoch: {}/{}.............'.format(epoch, EPOCHS), end = ' ')
            print("Train Loss: {:.4f}".format(train_loss[epoch-1]), end = ' ')
            print("Validation Loss: {:.4f}".format(val_loss[epoch-1]))
            
        
    return model,train_loss,val_loss


smaller batch size (2^)
epoch (early stopping)
residual net

In [112]:
def test_metrics(model, val_data, outcome = outcome):
    
    val_X = tensor_generator(val_data, outcome = outcome)[0]
    
    val_y = val_data.groupby('participant_id').first()[outcome].values.flatten()
    
    model.eval()
    

    out = model(val_X)
    predicted_props = out[:,0].detach().numpy()
    predicted_labels = 1*(predicted_props >0.5)
    
    
    acc = metrics.accuracy_score(val_y, predicted_labels, normalize=False) / float(val_y.size)
    confusion = metrics.confusion_matrix(val_y, predicted_labels)
    

    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    specificity = TN / (TN + FP)
    sensitivity = TP / (TP + FN)
    precision = TP/(TP + FP)
    
    auc = metrics.roc_auc_score(val_y, predicted_props)
    
    result_tab = pd.DataFrame(data = {'participant_id': list(val_data.groupby('participant_id').first().index),
                                     'labels':list(val_y),
                                     'predicted_labels':list(predicted_labels),
                                     'predicted_probs':list(predicted_props)})
    

    
    return acc,sensitivity,specificity,auc,precision,confusion,result_tab


In [113]:
import itertools

a = [[0.001,0.05], \
    [10,20,30], \
    [10,20,30], \
    [0.05,0.1,0.2]]



parameter_space = list(itertools.product(*a))

len(parameter_space)



54

In [114]:
def parameter_tuning(train_val_data, parameter_space,cv_time =5,seed = 999,outcome = outcome):
    
    best_parameter = tuple()
    best_auc = 0
    best_model = None
    
    train_val_data_ind = train_val_data.groupby('participant_id').first().copy()
    
    for param in parameter_space:
        
        auc_cv = []
        
        for i in range(cv_time):
            
            train_id = list(train_val_data_ind.groupby(outcome).sample(frac = 0.75, random_state = seed + i).sort_index(axis = 0).index)
            val_id =  list(np.setdiff1d(list(train_val_data_ind.index),train_id))
            
            train_data = train_val_data[train_val_data['participant_id'].isin(train_id)]
            val_data = train_val_data[train_val_data['participant_id'].isin(val_id)]
        
            model = train_epoch(train_data,val_data,learn_rate = param[0], \
                               hidden_dim1= param[1], hidden_dim2= param[2], \
                               drop_out= param[3], device = device)[0]
        
            auc = test_metrics(model, val_data, outcome = outcome)[3]
            
            auc_cv.append(auc)
        
        auc_ave = np.mean(auc_cv)
        
        if auc_ave >= best_auc:
            best_auc = auc_ave
            best_parameter = param
            best_model = model
            
    return best_parameter, best_model,best_auc
        
    

    

In [116]:
best_parameter, best_model,best_auc = parameter_tuning(train_val_data,parameter_space,cv_time =3,seed = 999,outcome = outcome)

Epoch: 10/500............. Train Loss: 0.6887 Validation Loss: 0.6875
Epoch: 20/500............. Train Loss: 0.6674 Validation Loss: 0.6656
Epoch: 30/500............. Train Loss: 0.6503 Validation Loss: 0.6490
Epoch: 40/500............. Train Loss: 0.6441 Validation Loss: 0.6432
Epoch: 50/500............. Train Loss: 0.6423 Validation Loss: 0.6414
Epoch: 60/500............. Train Loss: 0.6416 Validation Loss: 0.6407
Epoch: 70/500............. Train Loss: 0.6412 Validation Loss: 0.6404
Epoch: 80/500............. Train Loss: 0.6411 Validation Loss: 0.6402
Epoch: 90/500............. Train Loss: 0.6409 Validation Loss: 0.6400
Epoch: 100/500............. Train Loss: 0.6409 Validation Loss: 0.6400
Epoch: 110/500............. Train Loss: 0.6408 Validation Loss: 0.6399
Epoch: 120/500............. Train Loss: 0.6407 Validation Loss: 0.6399
Epoch: 130/500............. Train Loss: 0.6406 Validation Loss: 0.6399
Epoch: 140/500............. Train Loss: 0.6402 Validation Loss: 0.6401
Early stopping 

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_5632\3120863025.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6859 Validation Loss: 0.6851
Epoch: 20/500............. Train Loss: 0.6693 Validation Loss: 0.6679
Epoch: 30/500............. Train Loss: 0.6521 Validation Loss: 0.6508
Epoch: 40/500............. Train Loss: 0.6446 Validation Loss: 0.6437
Epoch: 50/500............. Train Loss: 0.6424 Validation Loss: 0.6415
Epoch: 60/500............. Train Loss: 0.6416 Validation Loss: 0.6407
Epoch: 70/500............. Train Loss: 0.6412 Validation Loss: 0.6404
Epoch: 80/500............. Train Loss: 0.6410 Validation Loss: 0.6402
Epoch: 90/500............. Train Loss: 0.6409 Validation Loss: 0.6401
Epoch: 100/500............. Train Loss: 0.6407 Validation Loss: 0.6401
Epoch: 110/500............. Train Loss: 0.6401 Validation Loss: 0.6403
Early stopping at 119


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_5632\3120863025.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.7279 Validation Loss: 0.7266
Epoch: 20/500............. Train Loss: 0.6970 Validation Loss: 0.6941
Epoch: 30/500............. Train Loss: 0.6632 Validation Loss: 0.6610
Epoch: 40/500............. Train Loss: 0.6486 Validation Loss: 0.6475
Epoch: 50/500............. Train Loss: 0.6443 Validation Loss: 0.6434
Epoch: 60/500............. Train Loss: 0.6428 Validation Loss: 0.6419
Epoch: 70/500............. Train Loss: 0.6420 Validation Loss: 0.6412
Epoch: 80/500............. Train Loss: 0.6416 Validation Loss: 0.6407
Epoch: 90/500............. Train Loss: 0.6414 Validation Loss: 0.6405
Epoch: 100/500............. Train Loss: 0.6412 Validation Loss: 0.6403
Epoch: 110/500............. Train Loss: 0.6411 Validation Loss: 0.6402
Epoch: 120/500............. Train Loss: 0.6410 Validation Loss: 0.6401
Epoch: 130/500............. Train Loss: 0.6409 Validation Loss: 0.6400
Epoch: 140/500............. Train Loss: 0.6409 Validation Loss: 0.6400
Epoch: 150/500.

...

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_5632\3120863025.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Epoch: 20/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Early stopping at 25


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_5632\3120863025.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Epoch: 20/500............. Train Loss: 0.6418 Validation Loss: 0.6406
Early stopping at 21


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_5632\3120863025.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [122]:
best_auc

0.6536971368531653

In [123]:
best_parameter

(0.001, 30, 10, 0.05)

In [124]:
test_acc,test_sensitivity,test_specificity, \
test_auc,test_precision,test_confusion,test_result_tab = \
test_metrics(best_model, test_data, outcome = outcome)


In [127]:
test_result_tab.to_csv('test_error_analysis.csv')

In [128]:
test_metrics(best_model, train_val_data, outcome = outcome)[6].to_csv('train_val_error_analysis.csv')

In [ ]:
# def project_cross_val(features,train_ratio = 0.7, learn_rate = 0.01,hidden_dim = 18, n_layers = 1, device = device, \
#                       batch_size = 1000,drop_prob = 0.2, EPOCHS = 1000, output_dim = 2, method = "RNN", \
#                       out_var = "was_preterm",label_smooth= True, multi_outcome= True):
    
#     ##check argument validity
#     if out_var != "was_preterm" and out_var != "was_early_preterm":
#         raise ValueError("outcome must be was_preterm or was_early_preterm")
    
#     ##obtain data by goal
#     if out_var == "was_preterm":
#         data = mydata_preterm
#     else:
#         data = mydata_epreterm
        
#     project_list = data['project'].unique()
#      ##accuracy, sen, spe, auc score
#     train_loss = {}
#     val_loss = {}
#     acc = {}
#     sen = {}
#     spe = {}
#     auc = {}
#     precision = {}
#     result_tabs = {}
    
    
#     for project in project_list:
#         train_list = data[data['project']!= project].groupby('participant_id').first().sort_index(axis = 0)
#         train_id = list(train_list.groupby(out_var).sample(frac = train_ratio, random_state = 100).sort_index(axis = 0).index)
#         val_id = np.setdiff1d(list(train_list.index),train_id)
#         test_id = list(data[data['project']== project].groupby('participant_id').first().sort_index(axis = 0).index)
#         train_x, train_y = tensor_generator(data,train_id,features = features, \
#                                             out_var= "was_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
#         val_x, val_y = tensor_generator(data,val_id,features = features, \
#                                             out_var= "was_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
        
#         train_dataloader = dataloader(train_x, train_y,batch_size =  batch_size)
#         val_dataloader = dataloader(val_x, val_y,batch_size =  batch_size)
#         model, train_loss[project],val_loss[project] = train_epoch(train_dataloader,val_dataloader, learn_rate = learn_rate, \
#                 hidden_dim = hidden_dim, n_layers = n_layers, drop_prob = drop_prob, \
#                 device = device, EPOCHS = EPOCHS, output_dim = output_dim, method = method)
        
#         acc[project],sen[project],spe[project],auc[project],precision[project],result_tabs[project] = \
#         test_metrics(model, data,test_id,features, out_var = out_var, multi_outcome = multi_outcome)
    
#     return train_loss,val_loss, result_tabs ,pd.DataFrame([acc,sen,spe,precision,auc], \
#                                                  index = ['accuracy','sensitivity','specificity','precision','AUC'])
    
   
    